In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import PIL
import json

In [ ]:
%matplotlib notebook

## Visualize Extracted Images

In [ ]:
path = os.path.expanduser("~/tbp/results/monty/monty_meets_world/")
num = '1'
depth = np.array(Image.open(path + f"depth{num}.jpg").transpose(PIL.Image.FLIP_TOP_BOTTOM))[:,:,0]
rgb = np.array(Image.open(path + f"rgb{num}.jpg").transpose(PIL.Image.FLIP_TOP_BOTTOM))
jet = np.array(Image.open(path + f"jet{num}.jpg").transpose(PIL.Image.FLIP_TOP_BOTTOM))

In [ ]:
depth_clipped = depth/255
depth_clipped[depth_clipped>0.6] = 1

In [ ]:
depth_clipped.shape

In [ ]:
plt.figure(figsize=(10,7))
plt.subplot(2,2,1)
plt.imshow(rgb)
plt.axis('off')
plt.title('RGB')
plt.subplot(2,2,2)
plt.imshow(jet)
plt.axis('off')
plt.title('JET from App')

plt.subplot(2,2,3)
plt.imshow(depth, cmap='jet_r')
plt.axis('off')
# plt.colorbar()
plt.title('Depth')
plt.subplot(2,2,4)
plt.imshow(depth_clipped, cmap='jet_r')
plt.axis('off')
# plt.colorbar()
plt.title('Depth Clipped')
plt.show()

In [ ]:
depthpng = np.array(Image.open(path + f"depthpng{3}.png").transpose(PIL.Image.FLIP_TOP_BOTTOM))[:,:,0]

In [ ]:
plt.figure()
plt.subplot(1,3,1)
plt.imshow(depthpng)
plt.axis('off')
plt.title('PNG')
plt.subplot(1,3,2)
plt.imshow(depth)
plt.axis('off')
plt.title('JPG')
plt.subplot(1,3,3)
plt.imshow(depth - depthpng)
plt.axis('off')
plt.title('Difference')
plt.tight_layout()
plt.show()

### Test JSON Saved Depth Points

In [ ]:
f = open(os.path.expanduser('~/Desktop/depth.json',))
data = json.load(f)

In [ ]:
plt.figure()
plt.plot(data)
plt.show()

In [ ]:
dataimg = np.array(data)
dataimg = dataimg.reshape((480, 640))

In [ ]:
plt.figure()
plt.imshow(dataimg, vmax=0.1)
plt.colorbar()
plt.show()

## Load Mug Observations

In [ ]:
obj_name = 'mug'
all_depth_imgs = []
all_rgb_imgs = []
for obs_id in range(4):
    depth = np.array(Image.open(path + f"{obj_name}/depth{obs_id}.png").transpose(PIL.Image.FLIP_TOP_BOTTOM))[:,:,0]
    depth_clipped = depth/255
    depth_clipped[depth_clipped>0.6] = 1
    all_depth_imgs.append(depth_clipped)
    rgb = np.array(Image.open(path + f"{obj_name}/rgb{obs_id}.jpg").transpose(PIL.Image.FLIP_TOP_BOTTOM))
    all_rgb_imgs.append(rgb)

In [ ]:
plt.figure(figsize=(7,10))
for obs_id in range(4):
    plt.subplot(4,2,(obs_id*2+1))
    plt.imshow(all_depth_imgs[obs_id])
    plt.axis('off')
    plt.subplot(4,2,(obs_id+1)*2)
    plt.imshow(all_rgb_imgs[obs_id])
    plt.axis('off')
plt.tight_layout()
plt.show()


## Turn Depth Image into Point Cloud

In [ ]:
from tbp.monty.frameworks.environment_utils.transforms import DepthTo3DLocations
from tbp.monty.frameworks.environment_utils.graph_utils import get_center_point_normal,get_curvature_at_point
import quaternion as qt

In [ ]:
agent_id = 'agent_01'
sensor_id = 'patch_01'
# depth = 1.2 - gaussian_filter(np.array(depth,dtype=float), sigma=0.5)
obs = {agent_id:{sensor_id:{"depth":depth_clipped}}}
rotation = qt.from_rotation_vector([np.pi / 2, 0.0, 0.0])
loc = np.array([0,0,0])
sensor_position = np.array([loc[0],loc[1],0])
state = {agent_id:{"sensors":{sensor_id + ".depth":{"rotation":rotation, 
                                                    "position":sensor_position}},
                  "rotation":rotation, "position":np.array([0,0,0])}}

In [ ]:
transform = DepthTo3DLocations(
                agent_id=agent_id,
                sensor_ids=[sensor_id],
                resolutions=[depth.shape],
                world_coord=True,
                zooms=1,
                get_all_points=True,
                use_semantic_sensor=False,
                depth_clip_sensors=(0,),
                clip_value=1.1,
            )


In [ ]:
obs = transform(obs, state=state)

In [ ]:
obs_3d = obs[agent_id][sensor_id]['semantic_3d']
locs = obs_3d[obs_3d[:,3] == 1]

# obs_dim = int(np.sqrt(obs_3d.shape[0]))
# half_obs_dim = obs_dim // 2
# center_id = half_obs_dim + obs_dim * half_obs_dim

# point_normal, _ = get_center_point_normal(
#                 obs_3d, patch_radius_frac=2.5
#             )
# k1, k2, dir1, dir2, _ = get_curvature_at_point(obs_3d, center_id, point_normal)

# center_loc = obs_3d[center_id,:3]

plt.figure()
ax = plt.subplot(1,1,1,projection='3d')
ax.scatter(locs[:,0], locs[:,1], locs[:,2], c=locs[:,2])
# ax.scatter(center_loc[0], center_loc[1], center_loc[2],s=50,c='green')
# linelen = 0.8
# colors = ['green','red','orange']
# for i,line in enumerate([point_normal, dir1, dir2]):
#     plt.plot([center_loc[0], center_loc[0] + line[0]*linelen], 
#              [center_loc[1], center_loc[1] + line[1]*linelen], 
#              [center_loc[2], center_loc[2] + line[2]*linelen],
#             color=colors[i])
# ax.set_aspect('equal')
ax.view_init(-90,-90)
plt.show()

## New Dataloaders etc.

In [ ]:
from tbp.monty.frameworks.utils.logging_utils import (load_stats,
                                                        print_overall_stats,
                                                        print_unsupervised_stats)
from tbp.monty.frameworks.utils.plot_utils import (plot_graph,
                                                         show_initial_hypotheses, 
                                                         plot_evidence_at_step,)

In [ ]:
log_path = os.path.expanduser("~/tbp/results/monty/projects/monty_runs/")
exp_name = "evidence_on_world_image/"
exp_path = log_path + exp_name
save_path = os.path.expanduser("~/tbp/results/monty/projects/monty_runs/") +exp_name+ 'stepwise_examples/'
train_stats, eval_stats, detailed_stats, lm_models = load_stats(exp_path,
                                                                load_train=True,
                                                                load_eval=True,
                                                                load_detailed=True,
                                                                pretrained_dict=None,
                                                               )

In [ ]:
train_stats

In [ ]:
eval_stats

### Plot Graph

In [ ]:
plot_graph(lm_models['1']['LM_0']['new_object0'], show_axticks=False)
# ax.grid(True)
plt.show()

### Plot Patch Depth Values

In [ ]:
print(lm_models['0']['LM_0']['new_object0'])

In [ ]:
step_id = 5
locs = np.array(detailed_stats['2']['SM_0']['raw_observations'][step_id]['depth'])
N = 64

# Plot depth image
xx, yy = np.meshgrid(np.arange(N), np.arange(N))

plt.figure(1)
ax = plt.subplot(1,1,1,projection='3d')
ax.scatter(xx, yy, locs)

# Plot 3D point cloud
locs = np.array(detailed_stats['2']['SM_0']['raw_observations'][step_id]['semantic_3d'])[:, :3]
plt.figure(2)
ax = plt.subplot(1,1,1,projection='3d')
ax.scatter(locs[:, 0], locs[:, 1], locs[:, 2])

### Plot LFS Observations & Save

In [ ]:
# Note that here the center_pixel_id is no more the 2D index of the patch center 
# relative to the 2D scene image, but rather the 3D loaction of the patch center 
# in the body-centric coordinate frame !!! Hence why the blue frame doesn't appear on the picture anymore.

episode = 1
num_steps = 41

for step in range(num_steps):
    plt.figure()
    plt.subplot(1,2,1)
    viewfinder_img = detailed_stats[str(episode)]['SM_1']['raw_observations'][step]['rgba']
    center_pixel_id = np.array(detailed_stats[str(episode)]['SM_0']['raw_observations'][step]['pixel_loc'])
    square_size = 64

    # Calculate top-left and bottom-right coordinates of the square
    # Careful here: x coordinates and y coordinates in pixel space are inverted
    # i.e. x is index 1, y is index 0
    x1 = center_pixel_id[0] - square_size // 2
    y1 = center_pixel_id[1] - square_size // 2
    x2 = center_pixel_id[0] + square_size // 2
    y2 = center_pixel_id[1] + square_size // 2
    image_plt = np.array(viewfinder_img.copy())

    # Set the outline pixels to blue
    image_plt[x1, y1:y2] = [0, 0, 255]
    image_plt[x2, y1:y2] = [0, 0, 255]
    image_plt[x1:x2, y1] = [0, 0, 255]
    image_plt[x1:x2, y2] = [0, 0, 255]
    plt.imshow(image_plt)
    plt.title('View Finder')
    plt.axis('off')
    plt.subplot(1,2,2)
    plt.imshow(detailed_stats[str(episode)]['SM_0']['raw_observations'][step]['rgba'])
    plt.title('Patch')
    plt.axis('off')
    plt.savefig(
            save_path + f"obs_{episode}_{step}.png",
            bbox_inches="tight",
        )

In [ ]:
episode = 1
objects = ["new_object0"]
for step in range(eval_stats['monty_matching_steps'][episode-1]):
    plot_evidence_at_step(detailed_stats,
                          lm_models,
                              episode, 
                              step,
                              objects,
                              is_touch_sensor=False,
                              is_2d_image_movement=True,
                              save_fig=True, 
                              save_path=save_path)

## Analyze Experiment on Pretrained Numenta Lab Models

In [ ]:
log_path = os.path.expanduser("~/tbp/results/monty/projects/monty_runs/")
pretrain_path = os.path.expanduser("~/tbp/results/monty/pretrained_models/")
pretrained_dict = pretrain_path + "pretrained_ycb_v4/touch_1lm_numenta_lab_obj/pretrained/"
exp_name = "world_image_on_scanned_model/"
# exp_name = "world_image_from_stream_on_scanned_model/"
exp_path = log_path + exp_name
save_path = os.path.expanduser("~/tbp/results/monty/projects/monty_runs/") + exp_name + 'stepwise_examples/'
train_stats, eval_stats, detailed_stats, lm_models = load_stats(exp_path,
                                                                load_train=False,
                                                                load_eval=True,
                                                                load_detailed=True,
                                                                pretrained_dict=pretrained_dict,
                                                               )

In [ ]:
print_overall_stats(eval_stats)

In [ ]:
eval_stats

In [ ]:
lm_models['pretrained'][0].keys()

In [ ]:
lm_models['pretrained'][0]['numenta_mug'].pos.shape

In [ ]:
lm_models['pretrained'][0]['ramen_pack'].pos.shape

In [ ]:
plot_graph(lm_models['pretrained'][0]['cocktail_bitters'], show_axticks=True)
plt.show()

In [ ]:
offset = 0.1
plt.figure()
ax = plt.subplot(1,1,1,projection='3d')
for i, object_id in enumerate(lm_models['pretrained'][0].keys()):
    pos = np.array(lm_models['pretrained'][0][object_id].pos)
    if i < 4:
        ax.scatter(pos[:,0] + offset * i, pos[:,1], pos[:,2], s=1)
    else:
        ax.scatter(pos[:,0] + offset * i/2, pos[:,1] + offset, pos[:,2], s=1)
ax.set_aspect("equal")
plt.show()

In [ ]:
object_name = 'numenta_mug'
lm_models = lm_models

fig = plt.figure()
ax = plt.subplot(1,1,1,projection='3d')
graph = lm_models['pretrained'][0][object_name]
model_pos = graph.pos
hsv_ids = lm_models['pretrained'][0][object_name].feature_mapping['patch']['hsv']
hue = graph.x[:,hsv_ids[0]:hsv_ids[0]+1]
colors = hue[:,0]
s = ax.scatter(
    model_pos[:, 0],
    model_pos[:, 1],
    model_pos[:, 2],
    s=5,
    vmin=0,
    vmax=1,
    cmap='hsv',
    c=colors
)
norm_len = 0.01
ax.set_box_aspect([1,1,1])
ax.set_xticks([]), ax.set_yticks([]), ax.set_zticks([])
fig.colorbar(s)
plt.title('HSV')
plt.show()

In [ ]:
object_name = 'numenta_mug'
lm_models = lm_models

fig = plt.figure()
ax = plt.subplot(1,1,1,projection='3d')
graph = lm_models['pretrained'][0][object_name]
model_pos = graph.pos
# hsv_ids = lm_models['pretrained'][0][object_name].feature_mapping['hsv']
curv_ids = lm_models['pretrained'][0][object_name].feature_mapping['patch']['principal_curvatures_log']
curv = graph.x[:,curv_ids[0]:curv_ids[1]]
colors = curv[:,0]
s = ax.scatter(
    model_pos[:, 0],
    model_pos[:, 1],
    model_pos[:, 2],
    s=5,
    vmin=-10,
    vmax=10,
    cmap='seismic',
    c=colors
)
norm_len = 0.01
ax.set_box_aspect([1,1,0.7])
ax.set_xticks([]), ax.set_yticks([]), ax.set_zticks([])
fig.colorbar(s)
plt.title("Principal Curvature Log")
plt.show()

In [ ]:
color_id = graph.feature_mapping['patch']['principal_curvatures_log'][0]
norm_len = 0.02

fig = plt.figure()
ax = plt.subplot(1,1,1,projection='3d')
s2 = ax.scatter(
    graph.pos[:, 0],
    graph.pos[:, 1],
    graph.pos[:, 2],
    s=5,
    alpha=0.5,
    c='blue'#np.arctan(np.array(sensor_graph.x[:,color_id])/100)
)
    
for p_id, p in enumerate(np.array(graph.pos)):
    norm = graph.norm[p_id]
    ax.plot([p[0], p[0] + norm[0] * norm_len],
            [p[1], p[1] + norm[1] * norm_len],
            [p[2], p[2] + norm[2] * norm_len],
            c='lightblue',
           )
    
# plt.legend(['From Mesh','From Sensor'])
plt.show()

In [ ]:
depth = np.array(detailed_stats[str(0)]["SM_0"]['raw_observations'][3]['semantic_3d'])

In [ ]:
detailed_stats[str(0)]["SM_0"]['processed_observations'][0]['features'].keys()

In [ ]:
np.array(pn) * 0.1

In [ ]:
episode = 0
step = 63
pn_len = 0.02

depth = np.array(detailed_stats[str(episode)]["SM_0"]['raw_observations'][step]['semantic_3d'])
pn = np.array(detailed_stats[str(episode)]["SM_0"]['processed_observations'][step]['features']['point_normal'])
pc1 = np.array(detailed_stats[str(episode)]["SM_0"]
               ['processed_observations'][step]['features']['curvature_directions'][:3])

plt.figure()
ax = plt.subplot(1,1,1,projection='3d')
on_obj = np.where(depth[:, 3])[0]
s = ax.scatter(
    depth[on_obj, 0],
    depth[on_obj, 1],
    depth[on_obj, 2],
    s=5,
    alpha=0.5,
    c=depth[on_obj, 3]
)

obs_dim = int(np.sqrt(depth.shape[0]))
half_obs_dim = obs_dim // 2
center_id = half_obs_dim + obs_dim * half_obs_dim
end_point = depth[center_id,:3] + pn * pn_len

ax.plot([depth[center_id, 0], end_point[0]],
       [depth[center_id, 1], end_point[1]],
       [depth[center_id, 2], end_point[2]],
       c='blue')
end_point = depth[center_id, :3] + pc1 * pn_len
ax.plot([depth[center_id, 0], end_point[0]],
       [depth[center_id, 1], end_point[1]],
       [depth[center_id, 2], end_point[2]],
       c='red')
plt.show()

In [ ]:
on_obj

In [ ]:
episode = 0
objects = ['numenta_mug', 'ramen_pack', 'chip_pack', 'cocktail_bitters']
for step in range(eval_stats['monty_matching_steps'][episode]):
    plot_evidence_at_step(detailed_stats,
                          lm_models,
                              episode, 
                              step,
                              objects,
                              is_touch_sensor=False,
                              is_2d_image_movement=True,
                              save_fig=True, 
                              save_path=save_path)

In [ ]:
depth = np.array(detailed_stats[str(episode)]["SM_1"]['raw_observations'][3]['depth'])

In [ ]:
plt.figure()
plt.imshow(depth)
plt.show()

In [ ]:
disps = np.array(detailed_stats[str(episode)]["LM_0"]['displacement'])
numg = np.array(lm_models['pretrained'][0]['numenta_mug'].pos)

In [ ]:
plt.figure()
ax = plt.subplot(1,1,1,projection='3d')
ax.scatter(numg[:,0], numg[:,1],numg[:,2], alpha=0.5)
loc = np.zeros(3)
for disp in disps:
    disp = disp
    next_loc = loc + disp
    ax.plot(
        [loc[0], next_loc[0]],
        [loc[1], next_loc[1]],
        [loc[2], next_loc[2]],
        c="black",
    )
    loc = next_loc
plt.show()

In [ ]:
a = np.array([0.0162, -0.0218])
b = np.array([-0.0327, -0.0601])


In [ ]:
np.linalg.norm(a-b)

In [ ]:
np.sum(np.linalg.norm(disps[:7], axis=1))

In [ ]:
disps

## Add Numenta Models to YCB Models

In [ ]:
import torch
import copy

In [ ]:
pretrain_path = os.path.expanduser("~/tbp/results/monty/pretrained_models/")
pretrained_dict_n = pretrain_path + "pretrained_ycb_v4/touch_1lm_numenta_lab_obj/pretrained/"
pretrained_dict_ycb = pretrain_path + "pretrained_ycb_v4/touch_1lm_10distinctobj/pretrained/"

In [ ]:
state_dict_n = torch.load(os.path.join(pretrained_dict_n, "model.pt"))
state_dict_ycb = torch.load(os.path.join(pretrained_dict_ycb, "model.pt"))

In [ ]:
def combine_models(sd1, sd2):
    combined_sd = copy.deepcopy(sd1)
    for model_id in sd2['lm_dict'][0]['graph_memory'].keys():
        combined_sd['lm_dict'][0]['target_to_graph_id'][model_id] = {model_id}
        combined_sd['lm_dict'][0]['graph_id_to_target'][model_id] = {model_id}
        combined_sd['lm_dict'][0]['graph_memory'][model_id] = sd2['lm_dict'][0]['graph_memory'][model_id]
    return combined_sd

In [ ]:
combined_models = combine_models(state_dict_ycb, state_dict_n)

In [ ]:
combined_models['lm_dict'][0]['graph_memory'].keys()

In [ ]:
save_path = pretrain_path + "pretrained_ycb_v4/touch_1lm_10ycb_3numetbp/pretrained/"
os.makedirs(save_path, exist_ok=True)
torch.save(combined_models, save_path + "model.pt")

In [ ]:
mug = np.array(combined_models['lm_dict'][0]['graph_memory']['mug'].pos)
nmug = np.array(combined_models['lm_dict'][0]['graph_memory']['numenta_mug'].pos)

In [ ]:
offset=0.1
plt.figure()
ax = plt.subplot(1,1,1,projection='3d')
ax.scatter(mug[:,0],mug[:,1],mug[:,2])
ax.scatter(nmug[:,0] + offset,nmug[:,1],nmug[:,2])
ax.set_aspect("equal")
plt.show()

In [ ]:
mustard = np.array(combined_models['lm_dict'][0]['graph_memory']['potted_meat_can'].pos)

In [ ]:
plt.figure()
offset=0.1
ax = plt.subplot(1,1,1,projection='3d')
ax.scatter(mustard[:,0],mustard[:,1],mustard[:,2])
ax.scatter(nmug[:,0] + offset,nmug[:,1],nmug[:,2])
ax.set_aspect("equal")
plt.show()

In [ ]:
log_path = os.path.expanduser("~/tbp/results/monty/projects/monty_runs/")
pretrain_path = os.path.expanduser("~/tbp/results/monty/pretrained_models/")
pretrained_dict = pretrain_path + "pretrained_ycb_v4/touch_1lm_10ycb_3numetbp/pretrained/"
exp_name = "world_image_on_10ycb_model/"
exp_path = log_path + exp_name
save_path = os.path.expanduser("~/tbp/results/monty/projects/monty_runs/") + exp_name + 'stepwise_examples/'
train_stats, eval_stats, detailed_stats, lm_models = load_stats(exp_path,
                                                                load_train=False,
                                                                load_eval=True,
                                                                load_detailed=True,
                                                                pretrained_dict=pretrained_dict,
                                                               )

In [ ]:
episode = 0
objects = ['numenta_mug', 'red_mug', 'potted_meat_can', 'mustard_bottle']
for step in range(eval_stats['monty_matching_steps'][episode]):
    plot_evidence_at_step(detailed_stats,
                          lm_models,
                              episode, 
                              step,
                              objects,
                              is_touch_sensor=False,
                              is_2d_image_movement=True,
                              save_fig=True, 
                              save_path=save_path)